In [ ]:
from src.plots.results import plot_params_by
datapath_list = [
    # '/home/dionisio35/Documents/thesis/results/data/mg17.0_0.1.json',
    # '/home/dionisio35/Documents/thesis/results/data/mg17.0_0.01.json',
    # '/home/dionisio35/Documents/thesis/results/data/mg17.0_0.1.json',
    # '/home/dionisio35/Documents/thesis/results/data/mg17.0_0.01.json',
    # '/home/dionisio35/Documents/thesis/results/data/mg30.0_0.1.json',
    # '/home/dionisio35/Documents/thesis/results/data/mg30.0_0.01.json',
    '/home/dionisio35/Documents/thesis/results/data/lorenz_0.1.json',
    '/home/dionisio35/Documents/thesis/results/data/lorenz_0.01.json',
]

datapath = '/home/dionisio35/Documents/thesis/results/data/ks35n64_0.1.json'
params = ['reservoir_sigma', 'rewiring', 'spectral_radius', 'leak_rate',]

plot_params_by(
    datapath=datapath,
    params=params,
    by_param='reservoir_degree',
)

In [ ]:
from src.plots.systems import plot_system
from src.utils import load_data

steps = 1
(
    _,
    t,
    _,
    _,
    _,
    _,
) = load_data(
    name='/home/dionisio35/Documents/thesis/data/Lorenz/Lorenz_dt0.02_steps150000_t-end3000.0_seed109482.csv',
    step=5,
)
t.shape
plot_system(
    target=t,
    length=1000,
    dt=0.1
)